### Покдлючение PySpark, загрузка библиотек: настройка изображений

In [82]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, countDistinct, count, size, when, regexp_extract, split

sns.set_style('darkgrid')
params = {'legend.fontsize' : 'medium', 'figure.figsize' : (10, 8), 'figure.dpi' : 100, 'axes.labelsize' : 'medium', 'axes.titlesize' : 'medium', 'xtick.labelsize' : 'medium', 'ytick.labelsize' : 'medium'}
plt.rcParams.update(params)

### Запуск сессии PySpark

In [83]:
# spark = SparkSession.builder.appName('EDA Films').getOrCreate()

### Загрузка и обзор датасетов

In [84]:
df_movies = spark.read.csv('sp_movies.csv', header=True, inferSchema=True)
df_ratings = spark.read.csv('sp_ratings.csv', header=True, inferSchema=True)
df_tags = spark.read.csv('sp_tags.csv', header=True, inferSchema=True)

In [85]:
df_movies.show(3)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows


In [86]:
df_ratings.show(3)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
+------+-------+------+---------+
only showing top 3 rows


In [87]:
df_tags.show(3)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
+------+-------+---------------+----------+
only showing top 3 rows


### Статистика данных

In [88]:
print('Кол-во пользователей поставивиших оценку:')
df_ratings.select(countDistinct('userId')).show()

Кол-во пользователей поставивиших оценку:
+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   610|
+----------------------+



In [89]:
print('Кол-во оцененных фильмов:')
df_ratings.select(countDistinct('movieId')).show()

Кол-во оцененных фильмов:
+-----------------------+
|count(DISTINCT movieId)|
+-----------------------+
|                   9724|
+-----------------------+



In [90]:
print('Кол-во фильмов c комментариями:')
df_tags.select(countDistinct('movieId')).show()

Кол-во фильмов c комментариями:
+-----------------------+
|count(DISTINCT movieId)|
+-----------------------+
|                   1572|
+-----------------------+



In [91]:
print('Кол-во комментариев к фильмам:')
df_tags.select(countDistinct('tag')).show()

Кол-во комментариев к фильмам:
+-------------------+
|count(DISTINCT tag)|
+-------------------+
|               1589|
+-------------------+



### Предобработка и настройка данных

In [92]:
df1 = df_ratings.alias('df1')
df2 = df_tags.alias('df2')
df3 = df_movies.alias('df3')

In [93]:
df3.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

### Извлечение года из названия

In [94]:
df3 = df3.withColumn('year', regexp_extract(df3['title'], r'\((\d{4})\)', 1))


In [95]:
df3.show()

+-------+--------------------+--------------------+----+
|movieId|               title|              genres|year|
+-------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Adventure|Animati...|1995|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|1995|
|      5|Father of the Bri...|              Comedy|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|
|      8| Tom and Huck (1995)|  Adventure|Children|1995|
|      9| Sudden Death (1995)|              Action|1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|
|     11|American Presiden...|Comedy|Drama|Romance|1995|
|     12|Dracula: Dead and...|       Comedy|Horror|1995|
|     13|        Balto (1995)|Adventure|Animati...|1995|
|     14|        Nixon (1995)|               Drama|1995|
|     15|Cutthroat Island ...|A

In [96]:
df_movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [97]:
from pyspark.sql import functions as F

# разбиваем строку по "|"
split_expr = F.split(F.col("genres"), "\\|")

# вычисляем реальное максимальное число жанров
max_genres = df3.select(F.max(F.size(split_expr))).first()[0]

# создаём новые колонки genre1..genreN (безопасно через try_element_at)
for i in range(1, max_genres + 1):
    df3 = df3.withColumn(f"genre{i}", F.expr(f"try_element_at(split(genres, '\\\\|'), {i})"))

# список новых колонок
genre_columns = [f"genre{i}" for i in range(1, max_genres + 1)]

# считаем количество непустых жанров
genre_count_expr = sum(
    F.when(F.col(c).isNotNull() & (F.col(c) != ""), 1).otherwise(0)
    for c in genre_columns
)
df3 = df3.withColumn("genre_count", genre_count_expr)

df3.show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+----+---------+---------+--------+------+--------+------+------+------+------+-------+-----------+
|movieId|title                                |genres                                     |year|genre1   |genre2   |genre3  |genre4|genre5  |genre6|genre7|genre8|genre9|genre10|genre_count|
+-------+-------------------------------------+-------------------------------------------+----+---------+---------+--------+------+--------+------+------+------+------+-------+-----------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|1995|Adventure|Animation|Children|Comedy|Fantasy |NULL  |NULL  |NULL  |NULL  |NULL   |5          |
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |1995|Adventure|Children |Fantasy |NULL  |NULL    |NULL  |NULL  |NULL  |NULL  |NULL   |3          |
|3      |Grumpier Old Men (1995)              |Com

In [98]:
df3 = df3.drop('genres')

In [99]:
df3.show()

+-------+--------------------+----+---------+---------+--------+------+--------+------+------+------+------+-------+-----------+
|movieId|               title|year|   genre1|   genre2|  genre3|genre4|  genre5|genre6|genre7|genre8|genre9|genre10|genre_count|
+-------+--------------------+----+---------+---------+--------+------+--------+------+------+------+------+-------+-----------+
|      1|    Toy Story (1995)|1995|Adventure|Animation|Children|Comedy| Fantasy|  NULL|  NULL|  NULL|  NULL|   NULL|          5|
|      2|      Jumanji (1995)|1995|Adventure| Children| Fantasy|  NULL|    NULL|  NULL|  NULL|  NULL|  NULL|   NULL|          3|
|      3|Grumpier Old Men ...|1995|   Comedy|  Romance|    NULL|  NULL|    NULL|  NULL|  NULL|  NULL|  NULL|   NULL|          2|
|      4|Waiting to Exhale...|1995|   Comedy|    Drama| Romance|  NULL|    NULL|  NULL|  NULL|  NULL|  NULL|   NULL|          3|
|      5|Father of the Bri...|1995|   Comedy|     NULL|    NULL|  NULL|    NULL|  NULL|  NULL|  N

### Анализ данных

In [100]:
rating_avg = df1.groupBy('movieId').agg(mean('rating').alias('rating_avg'))
rating_avg = rating_avg.withColumnRenamed('movieId', 'movieId_avg')
rating_avg.show(3)

+-----------+-----------------+
|movieId_avg|       rating_avg|
+-----------+-----------------+
|       1580|3.487878787878788|
|       2366|             3.64|
|       3175|             3.58|
+-----------+-----------------+
only showing top 3 rows


In [101]:
rating_count = df1.groupBy('movieId').agg(count('rating').alias('rating_count'))
rating_count = rating_count.withColumnRenamed('movieId', 'movieId_count')
rating_count.show(3)

+-------------+------------+
|movieId_count|rating_count|
+-------------+------------+
|         1580|         165|
|         2366|          25|
|         3175|          75|
+-------------+------------+
only showing top 3 rows


In [102]:
df_movie = rating_avg.join(rating_count, col('movieId_avg') == col('movieId_count'), 'inner').drop('movieId_count')
df_movie.show()

+-----------+------------------+------------+
|movieId_avg|        rating_avg|rating_count|
+-----------+------------------+------------+
|       1580| 3.487878787878788|         165|
|       2366|              3.64|          25|
|       3175|              3.58|          75|
|       1088| 3.369047619047619|          42|
|      32460|              4.25|           4|
|      44022| 3.217391304347826|          23|
|      96488|              4.25|           4|
|       1238| 4.055555555555555|           9|
|       1342|               2.5|          11|
|       1591|2.6346153846153846|          26|
|       1645| 3.411764705882353|          51|
|       4519|3.3333333333333335|           9|
|       2142|               2.7|          10|
|        471|              3.55|          40|
|       3997|1.8333333333333333|          12|
|        833|               2.0|           6|
|       3918|3.2777777777777777|           9|
|       7982|              3.25|           4|
|       1959|3.6666666666666665|  

In [103]:
user_rating = df1.groupBy('userId').agg(mean('rating').alias('user_rating_avg'))
user_rating = user_rating.withColumnRenamed('userId', 'userId_avg')
user_rating.show(3)

+----------+------------------+
|userId_avg|   user_rating_avg|
+----------+------------------+
|       148|3.7395833333333335|
|       463| 3.787878787878788|
|       471|             3.875|
+----------+------------------+
only showing top 3 rows


In [104]:
user_count = df1.groupBy('userId').agg(count('rating').alias('user_rating_count'))
user_count = user_count.withColumnRenamed('userId', 'userId_count')
user_count.show(3)

+------------+-----------------+
|userId_count|user_rating_count|
+------------+-----------------+
|         148|               48|
|         463|               33|
|         471|               28|
+------------+-----------------+
only showing top 3 rows


In [105]:
df_user = user_rating.join(user_count, col('userId_avg') == col('userId_count'), 'inner').drop('userId_count')
df_user.show()

+----------+------------------+-----------------+
|userId_avg|   user_rating_avg|user_rating_count|
+----------+------------------+-----------------+
|       148|3.7395833333333335|               48|
|       463| 3.787878787878788|               33|
|       471|             3.875|               28|
|       496| 3.413793103448276|               29|
|       243| 4.138888888888889|               36|
|       392|               3.2|               25|
|       540|               4.0|               42|
|        31|              3.92|               50|
|       516|3.6923076923076925|               26|
|        85|3.7058823529411766|               34|
|       137| 3.978723404255319|              141|
|       251| 4.869565217391305|               23|
|       451|3.7941176470588234|               34|
|       580| 3.529816513761468|              436|
|        65| 4.029411764705882|               34|
|       458|4.1525423728813555|               59|
|        53|               5.0|               20|


In [106]:
df_user.sort(col('user_rating_count').desc()).show()

+----------+------------------+-----------------+
|userId_avg|   user_rating_avg|user_rating_count|
+----------+------------------+-----------------+
|       414| 3.391957005189029|             2698|
|       599|2.6420500403551253|             2478|
|       474| 3.398956356736243|             2108|
|       448|2.8473712446351933|             1864|
|       274| 3.235884101040119|             1346|
|       610|3.6885560675883258|             1302|
|        68| 3.233730158730159|             1260|
|       380|3.6732348111658455|             1218|
|       606|3.6573991031390136|             1115|
|       288|3.1459715639810426|             1055|
|       249|3.6964627151051626|             1046|
|       387|3.2585199610516065|             1027|
|       182|3.5112589559877176|              977|
|       307|2.6656410256410257|              975|
|       603|3.5079533404029695|              943|
|       298| 2.363684771033014|              939|
|       177| 3.375553097345133|              904|


In [107]:
df_movie.sort(col('rating_count').desc()).show()

+-----------+------------------+------------+
|movieId_avg|        rating_avg|rating_count|
+-----------+------------------+------------+
|        356| 4.164133738601824|         329|
|        318| 4.429022082018927|         317|
|        296| 4.197068403908795|         307|
|        593| 4.161290322580645|         279|
|       2571| 4.192446043165468|         278|
|        260| 4.231075697211155|         251|
|        480|              3.75|         238|
|        110| 4.031645569620253|         237|
|        589| 3.970982142857143|         224|
|        527|             4.225|         220|
|       2959| 4.272935779816514|         218|
|          1|3.9209302325581397|         215|
|       1196|4.2156398104265405|         211|
|         50| 4.237745098039215|         204|
|       2858| 4.056372549019608|         204|
|         47|3.9753694581280787|         203|
|        780|3.4455445544554455|         202|
|        150| 3.845771144278607|         201|
|       1198|            4.2075|  

In [108]:
df_movie = df_movie.withColumnRenamed('movieId_avg', 'movieId')


In [109]:
dfk = df3.select('movieId', df3['genre1'])
df2 = df2.join(dfk, on='movieId', how='inner')

In [110]:
df2.show()

+-------+------+----------------+----------+---------+
|movieId|userId|             tag| timestamp|   genre1|
+-------+------+----------------+----------+---------+
|      1|   567|             fun|1525286013|Adventure|
|      1|   474|           pixar|1137206825|Adventure|
|      1|   336|           pixar|1139045764|Adventure|
|      2|   474|            game|1137375552|Adventure|
|      2|    62|  Robin Williams|1528843907|Adventure|
|      2|    62|magic board game|1528843932|Adventure|
|      2|    62|         fantasy|1528843929|Adventure|
|      3|   289|             old|1143424860|   Comedy|
|      3|   289|           moldy|1143424860|   Comedy|
|      5|   474|          remake|1137373903|   Comedy|
|      5|   474|       pregnancy|1137373903|   Comedy|
|      7|   474|          remake|1137375642|   Comedy|
|     11|   474|       president|1137374904|   Comedy|
|     11|   474|        politics|1137374904|   Comedy|
|     14|   474|       president|1137375623|    Drama|
|     14| 

In [112]:
user_tags = df2.groupBy('userId', 'movieId').agg(count('tag').alias('tag_count'))
max_tag_count = user_tags.agg({'tag_count': 'max'}).collect()[0][0]
user_tags_max = user_tags.filter(user_tags['tag_count'] == max_tag_count)

user_tags_max.show()

+------+-------+---------+
|userId|movieId|tag_count|
+------+-------+---------+
|   599|    296|      173|
+------+-------+---------+



In [113]:
user_tagcount = df2.groupBy('userId').agg(count('tag').alias('tag_count'))
user_tagcount.show()

+------+---------+
|userId|tag_count|
+------+---------+
|   513|        3|
|   193|       20|
|   300|        1|
|    76|        2|
|   606|        7|
|   103|        5|
|   336|       10|
|   319|        3|
|   599|      323|
|   177|        3|
|   132|        4|
|   305|        8|
|   291|        4|
|   435|        4|
|   274|        1|
|   474|     1507|
|   205|        3|
|   419|        5|
|   112|        3|
|   327|        7|
+------+---------+
only showing top 20 rows


In [114]:
movie_tagcount = df2.groupBy('movieId').agg(count('tag').alias('tag_count'))
cols = ['movieId', 'genre_count', 'genre1']
dfs = df3.select(cols)
movie_tagcount = movie_tagcount.join(dfs, on='movieId', how='inner')
movie_tagcount.show()

+-------+---------+-----------+---------+
|movieId|tag_count|genre_count|   genre1|
+-------+---------+-----------+---------+
|    471|        1|          1|   Comedy|
|   1088|        2|          3|    Drama|
|   1580|        1|          3|   Action|
|   1645|        1|          3|    Drama|
|   1959|        2|          2|    Drama|
|   2122|        1|          2|   Horror|
|   3175|        1|          3|Adventure|
|   6466|        1|          2|    Drama|
|   6620|        1|          2|   Comedy|
|   7833|        1|          3|   Comedy|
|   8638|        1|          2|    Drama|
|    540|        1|          1| Thriller|
|    858|        1|          2|    Crime|
|   1025|        2|          4|Animation|
|   1084|        2|          2|    Crime|
|   1721|        2|          2|    Drama|
|   2387|        2|          2|   Comedy|
|   3475|        1|          2|    Drama|
|   4190|        2|          1|    Drama|
|   6773|        1|          3|Animation|
+-------+---------+-----------+---

In [115]:
df_movie_renamed  = df_movie.withColumnRenamed('movieId_avg', 'movieId')

df3 = df3.join(df_movie_renamed, on='movieId', how='inner')
df3 = df3.withColumnRenamed('rating_avg', 'rating')
df3.show()

+-------+--------------------+----+-----------+---------+--------+--------+-------+------+------+------+------+-------+-----------+------------------+------------+
|movieId|               title|year|     genre1|   genre2|  genre3|  genre4| genre5|genre6|genre7|genre8|genre9|genre10|genre_count|            rating|rating_count|
+-------+--------------------+----+-----------+---------+--------+--------+-------+------+------+------+------+-------+-----------+------------------+------------+
|   1580|Men in Black (a.k...|1997|     Action|   Comedy|  Sci-Fi|    NULL|   NULL|  NULL|  NULL|  NULL|  NULL|   NULL|          3| 3.487878787878788|         165|
|   2366|    King Kong (1933)|1933|     Action|Adventure| Fantasy|  Horror|   NULL|  NULL|  NULL|  NULL|  NULL|   NULL|          4|              3.64|          25|
|   3175| Galaxy Quest (1999)|1999|  Adventure|   Comedy|  Sci-Fi|    NULL|   NULL|  NULL|  NULL|  NULL|  NULL|   NULL|          3|              3.58|          75|
|   1088|Dirty D

### Фильмы с самым высоким рейтингом (min голосов > X)

In [ ]:
df3.filter(col('rating_count') > 100) \
  .orderBy(col('rating').desc()) \
  .select('title', 'year', 'rating', 'rating_count') \
  .show(10, truncate=False)

+-----------------------------------------+----+-----------------+------------+
|title                                    |year|rating           |rating_count|
+-----------------------------------------+----+-----------------+------------+
|Shawshank Redemption, The (1994)         |1994|4.429022082018927|317         |
|Godfather, The (1972)                    |1972|4.2890625        |192         |
|Fight Club (1999)                        |1999|4.272935779816514|218         |
|Godfather: Part II, The (1974)           |1974|4.25968992248062 |129         |
|Departed, The (2006)                     |2006|4.252336448598131|107         |
|Goodfellas (1990)                        |1990|4.25             |126         |
|Dark Knight, The (2008)                  |2008|4.238255033557047|149         |
|Usual Suspects, The (1995)               |1995|4.237745098039215|204         |
|Princess Bride, The (1987)               |1987|4.232394366197183|142         |
|Star Wars: Episode IV - A New Hope (197

25/10/25 16:13:07 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1033366 ms exceeds timeout 120000 ms
25/10/25 16:13:07 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/25 16:13:09 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at 